<a href="https://colab.research.google.com/github/michaelgadda/LassoRegressionFromScratch/blob/main/LassoRegressionFromScratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from sklearn import datasets
from sklearn.linear_model import Lasso
from sklearn.linear_model import lasso_path
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
import numpy as np
from copy import deepcopy


In [56]:
class NewLassoRegression:
  def __init__(self, epochs=1000, lamb=1):
    self.epochs = epochs
    self.theta = None
    self.intercept = None
    self.lamb = lamb
    self.update_steps=None

  def preappend_bias_term(self, X):
    inter = np.ones(X.shape[0])
    X_new = np.column_stack((inter, X))
    return X_new



  def fit(self, X, Y):
    #X = X / np.sqrt(np.sum(np.square(X), axis=0)) <-- Pre-normalizing the data
    X = self.preappend_bias_term(X)

    Y = Y.reshape(-1,1)
    n_cols = X.shape[1]
    n_rows = X.shape[0]

    self.theta = np.zeros(n_cols).reshape(-1,1)
    self.update_steps = np.zeros((10000, n_cols))
    self.lamb = self.lamb * n_rows
    dw = np.zeros(n_cols)
    best_theta = np.zeros(n_cols)
    for epoch in range(self.epochs):

      y_pred = X @ self.theta
      for column in range(n_cols):
        X_col = X[:,column].reshape(-1,1)
        theta_j = deepcopy(self.theta[column])

        if column == 0:
          temp_theta = self.theta[column+1:].reshape(-1,1)
          temp_X = X[:,column+1:]
        elif column + 1 == n_cols:
          temp_theta = self.theta[:column].reshape(-1,1)
          temp_X = X[:, :column]
        else:
          temp_theta = np.vstack((self.theta[:column], self.theta[column+1:])).reshape(-1,1)
          temp_X = np.hstack((X[:, :column], X[:,column+1:]))

        y_pred = temp_X @ temp_theta
        rj = (Y - y_pred)
        theta_k = (X_col.T @ rj)
        if column != 0:
          if theta_k < -self.lamb:

            theta_k += self.lamb

          elif theta_k >= -self.lamb and theta_k <= self.lamb:

            theta_k = 0

          else: #theta_j > lambda

            theta_k -= self.lamb

        self.update_steps[epoch, column] = theta_k / ((X_col.T @ X_col))
        #Normalizing each update as we go instead of "pre"-normalizing the data
        self.theta[column] = theta_k / ((X_col.T @ X_col))

    self.theta[0] = np.sum(Y)/n_rows - self.theta[1:].T @ np.sum(X[:,1:], axis = 0)/n_rows

  def predict(self, X, theta=None):
      if theta is None:
        theta = self.theta
      return X@theta

In [54]:
diabetes_dataset = datasets.load_diabetes()

# Access the data and target variables
X = diabetes_dataset.data[:, :8]
y = diabetes_dataset.target.reshape(-1,1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 99)

In [55]:
#_, coef, _, n_iters = lasso_path(X_train, y_train, alphas=[1], return_n_iter = True ) <-- Lasso_path, using a different coordiante descent approach, it produced different results than linear_model.lasso
LR = Lasso(alpha=1, fit_intercept=True)
LR.fit(X_train, y_train)
myLR = NewLassoRegression(lamb=1, epochs=1000)
inter = np.ones(X_train.shape[0])
inter_2 = np.ones(X_test.shape[0])
X_new = np.column_stack((inter, X_train))
X_test_new = np.column_stack((inter_2, X_test))
myLR.fit(X_train, y_train)
sklasso_mse = mean_squared_error(LR.predict(X_test), y_test)
my_mse = mean_squared_error(myLR.predict(X_test_new), y_test)
print(f'\n\n SK: {[LR.coef_]}, \n\n MY: {myLR.intercept, myLR.theta} sklearn mse: {sklasso_mse} my_mse: {my_mse}')

<ipython-input-53-674024d7e161>:69: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.update_steps[epoch, column] = theta_k




 SK: [array([  0.        ,  -0.        , 471.13142551, 115.97936366,
         0.        ,   0.        , -26.11282509,   0.        ])], 

 MY: (None, array([[153.1279379 ],
       [  0.        ],
       [  0.        ],
       [471.12994968],
       [115.97926556],
       [  0.        ],
       [  0.        ],
       [-26.11335483],
       [  0.        ]])) sklearn mse: 3930.7268752441273 my_mse: 3930.7286010752905
